- DOING
    - 使用`./trial`目录下的txt文件模拟流程（可以按GNSS时间戳判断是否缺数据）
    - 使用提供的**trialname**测试
        - `T62288821d9d01`
        - `T62288821d9d02`

In [1]:
import sys
import requests
import numpy as np
import torch
import time
import copy
import importlib
from parse import parse

In [2]:
sys.path.append('./')
from spd_dnn import *
from data_struct import *
from ftr_generator import *
from coord_utils import *

# 算法流程

## 航向推算

### 积分

In [3]:
# def infer_bearing_via_integration(sec_gyro_unit_list, crnt_brng, collect_hz=250):
#     gy_ys = [u.sen_y for u in sec_gyro_unit_list]
#     bearing_diff = np.sum(gy_ys) / len(gy_ys) * -1.0
#     return (crnt_brng + bearing_diff) % 360.0


# def infer_bearing_via_model(sec_acc_unit_list, sec_gyro_unit_list, crnt_spd, crnt_brng, 
#                             model=None, ftr_gen=None, collect_hz=250):
#     sec_acc_unit_list = calibrate_sample_freq(sec_acc_unit_list, collect_hz=collect_hz)
#     sec_gyro_unit_lis = calibrate_sample_freq(sec_gyro_unit_list, collect_hz=collect_hz)
#     acc_ftr_list = ftr_gen.calc_imu_sec_ftr(sec_acc_unit_list)
#     gy_ftr_list = ftr_gen.calc_imu_sec_ftr(sec_gyro_unit_lis)

#     acc = torch.tensor([[acc_ftr_list]], dtype=torch.float).permute(0, 2, 1)
#     gy = torch.tensor([[gy_ftr_list]], dtype=torch.float).permute(0, 2, 1)
#     init_spd = torch.tensor([[[crnt_spd]]], dtype=torch.float)
#     init_brng = torch.tensor([[[crnt_brng]]], dtype=torch.float)
#     pred = model(acc, gy, init_spd=init_spd, init_brng=init_brng)
#     brng_diff = pred.cpu().detach().squeeze().numpy().tolist()[1]
    
#     return (crnt_brng + brng_diff) % 360


# def infer_bearing(sec_acc_unit_list, sec_gyro_unit_list, crnt_spd, crnt_brng, 
#                   model=None, ftr_gen=None, collect_hz=250):
#     integration_brng = infer_bearing_via_integration(sec_gyro_unit_list, crnt_brng, collect_hz=collect_hz)
    
#     return integration_brng


# def infer_bearing(sec_gyro_unit_list, crnt_brng, collect_hz=250, drift=0):
#     drift = -0.012397165517337367
#     gy_ys = [(u.sen_y - drift) for u in sec_gyro_unit_list]
#     bearing_diff = np.sum(gy_ys) / len(gy_ys) * -1.0
#     return (crnt_brng + bearing_diff) % 360.0

In [4]:
def project_gyro(obj, X=None, Y=None, Z=None):
    prj_ax = np.dot(np.array([obj.sen_x, obj.sen_y, obj.sen_z]), X)
    prj_ay = np.dot(np.array([obj.sen_x, obj.sen_y, obj.sen_z]), Y)
    prj_az = np.dot(np.array([obj.sen_x, obj.sen_y, obj.sen_z]), Z)

    prj_obj = copy.deepcopy(obj)
    prj_obj.gyr_x = prj_ax
    prj_obj.gyr_y = prj_ay
    prj_obj.gyr_z = prj_az

    return prj_obj


def infer_bearing(sec_gyro_unit_list, crnt_brng, collect_hz=250, drift=0):
    X = [-9.99832515e-01, 3.88289583e-04, -1.82972810e-02] 
    Y = [-0.01773261, -0.2678686, 0.96329226] 
    Z = [-0.00462569, 0.96195208, 0.27317907]
#     drift = -0.00032083975943054683
    drift = -0.011834270481663509
    
    prj_gyro_unit_list = [project_gyro(u, X=X, Y=Y, Z=Z) for u in sec_gyro_unit_list]
    gy_zs = [(u.gyr_z - drift) for u in prj_gyro_unit_list]
    bearing_diff = np.sum(gy_zs) / len(gy_zs) * -1.0
    return (crnt_brng + bearing_diff) % 360.0

## 速度推算（LSTM）

In [5]:
ftr_hz = 25
collect_hz = 250

model = load_spd_dnn(trained_model='./spd_dnn_weight_ftrHz%d.pt' % ftr_hz,
                     ftr_hz=ftr_hz, collect_hz=collect_hz)
model.eval()

ftr_gen = ftrGenerator(ftr_hz=ftr_hz, collect_hz=collect_hz)

Load trained model: ./spd_dnn_weight_ftrHz25.pt


In [6]:
def calibrate_sample_freq(imu_unit_list, collect_hz=250):
    if len(imu_unit_list) < collect_hz:
        imu_unit_list.extend([copy.deepcopy(imu_unit_list[-1])] * (collect_hz - len(imu_unit_list)))
    elif len(imu_unit_list) > collect_hz:
        imu_unit_list = imu_unit_list[-collect_hz:]
    else:
        pass
    return imu_unit_list


def infer_speed(sec_acc_unit_list, sec_gyro_unit_list, crnt_spd, crnt_brng,
                model=None, ftr_gen=None, collect_hz=250):
    sec_acc_unit_list = calibrate_sample_freq(sec_acc_unit_list, collect_hz=collect_hz)
    sec_gyro_unit_lis = calibrate_sample_freq(sec_gyro_unit_list, collect_hz=collect_hz)
    acc_ftr_list = ftr_gen.calc_imu_sec_ftr(sec_acc_unit_list)
    gy_ftr_list = ftr_gen.calc_imu_sec_ftr(sec_gyro_unit_lis)

    acc = torch.tensor([[acc_ftr_list]], dtype=torch.float).permute(0, 2, 1)
    gy = torch.tensor([[gy_ftr_list]], dtype=torch.float).permute(0, 2, 1)
    init_spd = torch.tensor([[[crnt_spd]]], dtype=torch.float)
    init_brng = torch.tensor([[[crnt_brng]]], dtype=torch.float)
    pred = model(acc, gy, init_spd=init_spd, init_brng=init_brng)
    spd_diff = pred.cpu().detach().squeeze().numpy().tolist()[0]

    return max(0.0, min(15.0, crnt_spd + spd_diff))

## 速度推算（OBD）

In [7]:
# def infer_speed_via_obd(obd_unit_list, crnt_spd=None):
#     obd_spds = [u.vehicle_speed for u in obd_unit_list]
#     spd = np.sum(obd_spds) / len(obd_spds) * 1.0
#     return max(-1, min(15, spd * 0.8))


def infer_speed_via_obd(obd_unit_list, crnt_spd=None):
    obd_spds = [u.vehicle_speed for u in obd_unit_list]
    spd = np.sum(obd_spds) / len(obd_spds) * 1.0
    return max(-1, min(15, spd * 0.8))

## 速度推算（积分）

In [8]:
def project_acce(obj, X=None, Y=None, Z=None):
    prj_ax = np.dot(np.array([obj.sen_x, obj.sen_y, obj.sen_z]), X)
    prj_ay = np.dot(np.array([obj.sen_x, obj.sen_y, obj.sen_z]), Y)
    prj_az = np.dot(np.array([obj.sen_x, obj.sen_y, obj.sen_z]), Z)

    prj_obj = copy.deepcopy(obj)
    prj_obj.acc_x = prj_ax
    prj_obj.acc_y = prj_ay
    prj_obj.acc_z = prj_az

    return prj_obj


def infer_speed_diff_via_integration(acce_unit_list, crnt_spd):
    X = [-9.99832515e-01, 3.88289583e-04, -1.82972810e-02] 
    Y = [-0.01773261, -0.2678686, 0.96329226] 
    Z = [-0.00462569, 0.96195208, 0.27317907]
#     drift = 0.06431939105769646
    drift = 0.054283966428103465
    
    prj_acce_unit_list = [project_acce(u, X=X, Y=Y, Z=Z) for u in acce_unit_list]
    prj_ys = [(u.acc_y - drift) for u in prj_acce_unit_list]
    spd_diff = np.sum(prj_ys) / len(prj_ys) * -1.0
    return spd_diff

## 速度推算（加权）

In [9]:
def infer_speed_via_multi(acce_unit_list, obd_unit_list, crnt_spd):
    acce_spd_diff = infer_speed_diff_via_integration(acce_unit_list, crnt_spd)
    obd_spd = infer_speed_via_obd(obd_unit_list)
    
    if obd_spd < 1e-4 and acce_spd_diff < 0:
        spd = acce_spd_diff
    else:
        spd = obd_spd
    return spd

# 服务端配置及请求

- 正式比赛配置

In [10]:
# server = 'http://121.196.218.26/evaalapi/'
# trialname = 'S622team708_02'

# estfmt = "{pts:.3f},{c:.3f},{h:.3f},{s:.3f},{pos}"
# statefmt = "{trialts:.3f},{rem:.3f},{V:.3f},{S:.3f},{p:.3f},{h:.3f},{pts:.3f},{pos}"

- 使用官方服务器测试

In [11]:
# server = 'http://121.196.218.26/evaalapi/'
# trialname = 'T623412f0da401'

# estfmt = "{pts:.3f},{c:.3f},{h:.3f},{s:.3f},{pos}"
# statefmt = "{trialts:.3f},{rem:.3f},{V:.3f},{S:.3f},{p:.3f},{h:.3f},{pts:.3f},{pos}"

- 使用本地服务器测试

In [12]:
server = 'http://127.0.0.1:5000/evaalapi/'
trialname = 'demo2023'

estfmt = "{pts:.3f},{c:.3f},{h:.3f},{s:.3f},{pos}"
statefmt = "{trialts:.3f},{rem:.3f},{V:.3f},{S:.3f},{p:.3f},{h:.3f},{pts:.3f},{pos}"

In [13]:
def do_state():
    r = requests.get(server + trialname + '/state')
    print(r.text)
    s = parse(statefmt, r.text)
    return s.named
    
    
def do_reload():
    r = requests.get(server + trialname + '/reload')
    s = parse(statefmt, r.text)
    print(s.named)

In [14]:
def parse_response(text):
    sen_str_list = text.splitlines()
    acc_unit_list, gy_unit_list = [], []
    obd_unit_list = []
    gnss_unit = None
    for sen_str in sen_str_list:
        if sen_str.startswith('ACCE'):
            acc_unit_list.append(acceUnit(sen_str))
        elif sen_str.startswith('GYRO'):
            gy_unit_list.append(gyroUnit(sen_str))
        elif sen_str.startswith('GNSS'):
            gnss_unit = gnssUnit(sen_str)
        elif sen_str.startswith('OBD'):
            obd_unit_list.append(obdUnit(sen_str))
        else:
            continue
    return acc_unit_list, gy_unit_list, obd_unit_list, gnss_unit


def estimate_next_loc(acc_unit_list, gy_unit_list, obd_unit_list, gnss_unit, 
                      model=None, ftr_gen=None, ftr_hz=50, collect_hz=250):
    crnt_brng = gnss_unit.bearing
    crnt_spd = gnss_unit.loc_speed
    if crnt_spd < 0:
        crnt_spd = gnss_unit.speed
    
    next_brng = infer_bearing(gy_unit_list, crnt_brng, 
                              collect_hz=collect_hz)
    
    if len(obd_unit_list) > 10:
#         next_spd = infer_speed_via_obd(obd_unit_list)
        next_spd = infer_speed_via_multi(acc_unit_list, obd_unit_list, crnt_spd)
    else:
        next_spd = infer_speed(acc_unit_list, gy_unit_list, crnt_spd, crnt_brng, 
                               model=model, ftr_gen=ftr_gen, collect_hz=collect_hz)

    next_lng, next_lat = geo_util.position_from_angle_and_dist(gnss_unit.lng, gnss_unit.lat, 
                                                               next_brng, next_spd)
    return next_lng, next_lat, next_spd, next_brng


def estimate_next_loc_by_pred(acc_unit_list, gy_unit_list, obd_unit_list, pred_loc, 
                              model=None, ftr_gen=None, ftr_hz=25, collect_hz=250):
    crnt_lng, crnt_lat, crnt_spd, crnt_brng = pred_loc
    
    next_brng = infer_bearing(gy_unit_list, crnt_brng, 
                              collect_hz=collect_hz)
    
    if len(obd_unit_list) > 10:
#         next_spd = infer_speed_via_obd(obd_unit_list)
        next_spd = infer_speed_via_multi(acc_unit_list, obd_unit_list, crnt_spd)
    else:
        next_spd = infer_speed(acc_unit_list, gy_unit_list, crnt_spd, crnt_brng, 
                               model=model, ftr_gen=ftr_gen, collect_hz=collect_hz)
    
    next_lng, next_lat = geo_util.position_from_angle_and_dist(crnt_lng, crnt_lat, 
                                                               next_brng, next_spd)
    return next_lng, next_lat, next_spd, next_brng
    
    
def estimate_next_loc_by_dft(crnt_lng, crnt_lat, crnt_spd, crnt_brng):
    next_brng, next_spd = crnt_brng, crnt_spd
    next_lng, next_lat = geo_util.position_from_angle_and_dist(crnt_lng, crnt_lat, 
                                                               next_brng, next_spd)
    return next_lng, next_lat, next_spd, next_brng
    

In [15]:
# do_reload()

In [16]:
# s = do_state()
# print(s)

In [17]:
# r = requests.get(server + trialname + '/estimates')
# print(r.text)

In [18]:
# r = requests.get(server + trialname + '/nextdata?horizon=1.0&position=0,0,0')
# print(r.text)

# r = requests.get(server + trialname + '/log')
# print(r.text)

In [19]:
# 注意time.sleep

In [20]:
init_pos = None  # 车辆初始位置
altitude = 0.0  # 高程
pre_gnss = None  # 上一时刻GNSS
pred_loc = None
next_position = None
gt_gnss_unit_list = []

sec_cnt = 0
request_arr = []

do_reload()  # 正式比赛需要注释
while True:
    s = do_state()
    if s['trialts'] == 0.0:
        print('testing start...')
        try:
            lat, lng, altitude = [float(x) for x in s['pos'].split(',')]
        except ValueError as e:
            lat, lng, altitude = [float(x) for x in s['pos'].split(';')]
        init_pos = [lng, lat]
    elif s['trialts'] < 0:
        print('testing timeout...')
        break
    else:
        pass
    
    if next_position is None:
        r = requests.get(server + trialname + '/nextdata?horizon=1.0')
    else:
        r = requests.get(server + trialname + \
                         '/nextdata?horizon=1.0&position=%s,%.1f' % (next_position, altitude))
    
    acc_unit_list, gy_unit_list, obd_unit_list, gnss_unit = parse_response(r.text)
    
#     request_arr.append(r.text.splitlines())
    if gnss_unit is None:
        print(len(acc_unit_list))
        print(len(gy_unit_list))
        
    if gnss_unit is not None:
        gt_gnss_unit_list.append(gnss_unit)
        
    if len(acc_unit_list) < 10 and gnss_unit is None:
        if pre_gnss is not None:
            lng, lat, spd, brng = pre_gnss.lng, pre_gnss.lat, pre_gnss.loc_speed, pre_gnss.bearing
            if spd < 0:
                spd = pre_gnss.speed
            next_lng, next_lat, \
            next_spd, next_brng = estimate_next_loc_by_dft(lng, lat, spd, brng)
            next_position = '%.6f,%.6f' % (next_lng, next_lat)
            pred_loc = [next_lng, next_lat, next_spd, next_brng]
        elif pred_loc is not None:
            next_lng, next_lat, \
            next_spd, next_brng = estimate_next_loc_by_dft(pred_loc[0], pred_loc[1], 
                                                           pred_loc[2], pred_loc[3])
            next_position = '%.6f,%.6f' % (next_lng, next_lat)
            pred_loc = [next_lng, next_lat, next_spd, next_brng]
        else:
            pass
        time.sleep(0.501)
        sec_cnt += 1
        continue
    
    if gnss_unit is not None:
        gnss_unit.pre_gnss = pre_gnss
        pre_gnss = gnss_unit
        next_position = '%.6f,%.6f' % (gnss_unit.lng, gnss_unit.lat)
    elif pre_gnss is not None:
        next_lng, next_lat, \
        next_spd, next_brng = estimate_next_loc(acc_unit_list, gy_unit_list, obd_unit_list, pre_gnss, 
                                                model=model, ftr_gen=ftr_gen, 
                                                ftr_hz=ftr_hz, collect_hz=collect_hz)
        next_position = '%.6f,%.6f' % (next_lng, next_lat)
        pred_loc = [next_lng, next_lat, next_spd, next_brng]
        pre_gnss = None
    elif pred_loc is not None:
        next_lng, next_lat, \
        next_spd, next_brng = estimate_next_loc_by_pred(acc_unit_list, gy_unit_list, obd_unit_list, pred_loc, 
                                                        model=model, ftr_gen=ftr_gen, 
                                                        ftr_hz=ftr_hz, collect_hz=collect_hz)
        next_position = '%.6f,%.6f' % (next_lng, next_lat)
        pred_loc = [next_lng, next_lat, next_spd, next_brng]
    else:
        pass
        
#     time.sleep(0.101)
    sec_cnt += 1
#     if sec_cnt == 2500:
#         break

{'trialts': 0.0, 'rem': -1.0, 'V': 3.0, 'S': 15.0, 'p': 0.0, 'h': 0.0, 'pts': 0.0, 'pos': '40.002201,116.385107,91.500'}
0.000,-1.000,3.000,15.000,0.000,0.000,0.000,40.002201,116.385107,91.500
testing start...
35.680,17.989,3.000,15.000,1694514488.581,1.000,34.680,40.002201,116.385107,91.500
36.680,17.986,3.000,15.000,1694514488.594,1.000,35.680,116.385107,40.002201,91.5
37.680,17.984,3.000,15.000,1694514488.612,1.000,36.680,116.385109,40.002200,91.5
38.680,17.982,3.000,15.000,1694514488.633,1.000,37.680,116.385114,40.002203,91.5
39.680,17.982,3.000,15.000,1694514488.654,1.000,38.680,116.385118,40.002208,91.5
40.680,17.984,3.000,15.000,1694514488.673,1.000,39.680,116.385127,40.002211,91.5
41.680,17.984,3.000,15.000,1694514488.691,1.000,40.680,116.385134,40.002213,91.5
42.680,17.984,3.000,15.000,1694514488.709,1.000,41.680,116.385135,40.002213,91.5
43.680,17.986,3.000,15.000,1694514488.728,1.000,42.680,116.385136,40.002212,91.5
44.680,17.983,3.000,15.000,1694514488.744,1.000,43.680,116.

136.680,17.968,3.000,15.000,1694514490.877,1.000,135.680,116.386296,40.003482,91.5
137.680,17.970,3.000,15.000,1694514490.912,1.000,136.680,116.386358,40.003484,91.5
138.680,17.968,3.000,15.000,1694514490.943,1.000,137.680,116.386410,40.003486,91.5
139.680,17.968,3.000,15.000,1694514490.977,1.000,138.680,116.386458,40.003488,91.5
140.680,17.971,3.000,15.000,1694514491.010,1.000,139.680,116.386506,40.003489,91.5
141.680,17.973,3.000,15.000,1694514491.040,1.000,140.680,116.386555,40.003492,91.5
142.680,17.971,3.000,15.000,1694514491.069,1.000,141.680,116.386603,40.003496,91.5
143.680,17.968,3.000,15.000,1694514491.101,1.000,142.680,116.386656,40.003499,91.5
144.680,17.971,3.000,15.000,1694514491.134,1.000,143.680,116.386709,40.003502,91.5
145.680,17.971,3.000,15.000,1694514491.165,1.000,144.680,116.386764,40.003505,91.5
146.680,17.966,3.000,15.000,1694514491.196,1.000,145.680,116.386822,40.003507,91.5
147.680,17.970,3.000,15.000,1694514491.234,1.000,146.680,116.386874,40.003508,91.5
148.

236.680,17.965,3.000,15.000,1694514494.356,1.000,235.680,116.390303,40.001525,91.5
237.680,17.965,3.000,15.000,1694514494.393,1.000,236.680,116.390299,40.001450,91.5
238.680,17.970,3.000,15.000,1694514494.429,1.000,237.680,116.390295,40.001388,91.5
239.680,17.970,3.000,15.000,1694514494.464,1.000,238.680,116.390292,40.001341,91.5
240.680,17.968,3.000,15.000,1694514494.496,1.000,239.680,116.390292,40.001304,91.5
241.680,17.968,3.000,15.000,1694514494.529,1.000,240.680,116.390287,40.001277,91.5
242.680,17.967,3.000,15.000,1694514494.562,1.000,241.680,116.390285,40.001258,91.5
243.680,17.962,3.000,15.000,1694514494.596,1.000,242.680,116.390288,40.001243,91.5
244.680,17.964,3.000,15.000,1694514494.635,1.000,243.680,116.390290,40.001236,91.5
245.680,17.958,3.000,15.000,1694514494.672,1.000,244.680,116.390294,40.001235,91.5
246.680,17.963,3.000,15.000,1694514494.716,1.000,245.680,116.390301,40.001233,91.5
247.680,17.962,3.000,15.000,1694514494.755,1.000,246.680,116.390309,40.001233,91.5
248.

338.680,17.970,3.000,15.000,1694514497.850,1.000,337.680,116.390854,39.997426,91.5
339.680,17.971,3.000,15.000,1694514497.881,1.000,338.680,116.390865,39.997338,91.5
340.680,17.971,3.000,15.000,1694514497.911,1.000,339.680,116.390877,39.997257,91.5
341.680,17.972,3.000,15.000,1694514497.942,1.000,340.680,116.390886,39.997187,91.5
342.680,17.970,3.000,15.000,1694514497.971,1.000,341.680,116.390894,39.997128,91.5
343.680,17.971,3.000,15.000,1694514498.003,1.000,342.680,116.390906,39.997076,91.5
344.680,17.974,3.000,15.000,1694514498.034,1.000,343.680,116.390920,39.997030,91.5
345.680,17.973,3.000,15.000,1694514498.061,1.000,344.680,116.390932,39.996987,91.5
346.680,17.969,3.000,15.000,1694514498.090,1.000,345.680,116.390944,39.996950,91.5
347.680,17.968,3.000,15.000,1694514498.122,1.000,346.680,116.390954,39.996923,91.5
348.680,17.967,3.000,15.000,1694514498.156,1.000,347.680,116.390959,39.996907,91.5
349.680,17.970,3.000,15.000,1694514498.191,1.000,348.680,116.390958,39.996896,91.5
350.

443.680,17.968,3.000,15.000,1694514501.038,1.000,442.680,116.386666,39.996658,91.5
444.680,17.972,3.000,15.000,1694514501.072,1.000,443.680,116.386564,39.996657,91.5
445.680,17.972,3.000,15.000,1694514501.101,1.000,444.680,116.386460,39.996655,91.5
446.680,17.972,3.000,15.000,1694514501.131,1.000,445.680,116.386350,39.996653,91.5
447.680,17.972,3.000,15.000,1694514501.160,1.000,446.680,116.386242,39.996652,91.5
448.680,17.924,3.000,15.000,1694514501.190,1.000,447.680,116.386135,39.996650,91.5
449.680,17.970,3.000,15.000,1694514501.267,1.000,448.680,116.386030,39.996644,91.5
450.680,17.971,3.000,15.000,1694514501.299,1.000,449.680,116.385932,39.996640,91.5
451.680,17.972,3.000,15.000,1694514501.329,1.000,450.680,116.385840,39.996636,91.5
452.680,17.972,3.000,15.000,1694514501.359,1.000,451.680,116.385757,39.996631,91.5
453.680,17.970,3.000,15.000,1694514501.388,1.000,452.680,116.385682,39.996628,91.5
454.680,17.973,3.000,15.000,1694514501.420,1.000,453.680,116.385612,39.996625,91.5
455.

...

5037.680,17.970,3.000,15.000,1694514647.599,1.000,5036.680,116.381092,39.993042,91.5
5038.680,17.972,3.000,15.000,1694514647.631,1.000,5037.680,116.381089,39.993124,91.5
5039.680,17.972,3.000,15.000,1694514647.660,1.000,5038.680,116.381087,39.993212,91.5
5040.680,17.973,3.000,15.000,1694514647.689,1.000,5039.680,116.381084,39.993307,91.5
5041.680,17.973,3.000,15.000,1694514647.717,1.000,5040.680,116.381078,39.993408,91.5
5042.680,17.974,3.000,15.000,1694514647.746,1.000,5041.680,116.381074,39.993511,91.5
5043.680,17.974,3.000,15.000,1694514647.773,1.000,5042.680,116.381069,39.993615,91.5
5044.680,17.974,3.000,15.000,1694514647.800,1.000,5043.680,116.381060,39.993717,91.5
5045.680,17.972,3.000,15.000,1694514647.828,1.000,5044.680,116.381047,39.993819,91.5
5046.680,17.972,3.000,15.000,1694514647.857,1.000,5045.680,116.381033,39.993923,91.5
5047.680,17.970,3.000,15.000,1694514647.887,1.000,5046.680,116.381021,39.994032,91.5
5048.680,17.973,3.000,15.000,1694514647.918,1.000,5047.680,116.38

5141.680,17.970,3.000,15.000,1694514650.466,1.000,5140.680,116.386640,39.996583,91.5
5142.680,17.971,3.000,15.000,1694514650.498,1.000,5141.680,116.386685,39.996583,91.5
5143.680,17.973,3.000,15.000,1694514650.528,1.000,5142.680,116.386732,39.996584,91.5
5144.680,17.975,3.000,15.000,1694514650.556,1.000,5143.680,116.386768,39.996586,91.5
5145.680,17.976,3.000,15.000,1694514650.583,1.000,5144.680,116.386790,39.996586,91.5
5146.680,17.976,3.000,15.000,1694514650.609,1.000,5145.680,116.386792,39.996585,91.5
5147.680,17.976,3.000,15.000,1694514650.634,1.000,5146.680,116.386785,39.996583,91.5
5148.680,17.973,3.000,15.000,1694514650.659,1.000,5147.680,116.386777,39.996582,91.5
5149.680,17.973,3.000,15.000,1694514650.687,1.000,5148.680,116.386772,39.996583,91.5
5150.680,17.975,3.000,15.000,1694514650.716,1.000,5149.680,116.386768,39.996583,91.5
5151.680,17.975,3.000,15.000,1694514650.743,1.000,5150.680,116.386768,39.996583,91.5
5152.680,17.976,3.000,15.000,1694514650.769,1.000,5151.680,116.38

5244.680,17.975,3.000,15.000,1694514653.312,1.000,5243.680,116.389703,39.996743,91.5
5245.680,17.973,3.000,15.000,1694514653.339,1.000,5244.680,116.389602,39.996745,91.5
5246.680,17.972,3.000,15.000,1694514653.367,1.000,5245.680,116.389498,39.996745,91.5
5247.680,17.971,3.000,15.000,1694514653.396,1.000,5246.680,116.389396,39.996744,91.5
5248.680,17.971,3.000,15.000,1694514653.429,1.000,5247.680,116.389299,39.996742,91.5
5249.680,17.973,3.000,15.000,1694514653.460,1.000,5248.680,116.389215,39.996738,91.5
5250.680,17.974,3.000,15.000,1694514653.488,1.000,5249.680,116.389152,39.996735,91.5
5251.680,17.970,3.000,15.000,1694514653.516,1.000,5250.680,116.389109,39.996735,91.5
5252.680,17.975,3.000,15.000,1694514653.548,1.000,5251.680,116.389089,39.996735,91.5
5253.680,17.976,3.000,15.000,1694514653.575,1.000,5252.680,116.389091,39.996732,91.5
5254.680,17.974,3.000,15.000,1694514653.600,1.000,5253.680,116.389099,39.996727,91.5
5255.680,17.976,3.000,15.000,1694514653.627,1.000,5254.680,116.38

5347.680,17.973,3.000,15.000,1694514656.143,1.000,5346.680,116.386053,39.996626,91.5
5348.680,17.974,3.000,15.000,1694514656.171,1.000,5347.680,116.385971,39.996622,91.5
5349.680,17.971,3.000,15.000,1694514656.199,1.000,5348.680,116.385895,39.996617,91.5
5350.680,17.974,3.000,15.000,1694514656.229,1.000,5349.680,116.385824,39.996613,91.5
5351.680,17.973,3.000,15.000,1694514656.257,1.000,5350.680,116.385766,39.996610,91.5
5352.680,17.971,3.000,15.000,1694514656.286,1.000,5351.680,116.385723,39.996606,91.5
5353.680,17.974,3.000,15.000,1694514656.316,1.000,5352.680,116.385691,39.996604,91.5
5354.680,17.975,3.000,15.000,1694514656.345,1.000,5353.680,116.385663,39.996602,91.5
5355.680,17.974,3.000,15.000,1694514656.372,1.000,5354.680,116.385644,39.996602,91.5
5356.680,17.975,3.000,15.000,1694514656.400,1.000,5355.680,116.385643,39.996602,91.5
5357.680,17.975,3.000,15.000,1694514656.426,1.000,5356.680,116.385645,39.996603,91.5
5358.680,17.974,3.000,15.000,1694514656.453,1.000,5357.680,116.38

5447.680,17.972,3.000,15.000,1694514658.993,1.000,5446.680,116.385205,40.000479,91.5
5448.680,17.972,3.000,15.000,1694514659.022,1.000,5447.680,116.385203,40.000479,91.5
5449.680,17.975,3.000,15.000,1694514659.051,1.000,5448.680,116.385202,40.000480,91.5
5450.680,17.976,3.000,15.000,1694514659.080,1.000,5449.680,116.385201,40.000485,91.5
5451.680,17.974,3.000,15.000,1694514659.106,1.000,5450.680,116.385199,40.000485,91.5
5452.680,17.975,3.000,15.000,1694514659.134,1.000,5451.680,116.385199,40.000484,91.5
5453.680,17.976,3.000,15.000,1694514659.160,1.000,5452.680,116.385199,40.000484,91.5
5454.680,17.975,3.000,15.000,1694514659.186,1.000,5453.680,116.385199,40.000484,91.5
5455.680,17.975,3.000,15.000,1694514659.212,1.000,5454.680,116.385199,40.000484,91.5
5456.680,17.977,3.000,15.000,1694514659.240,1.000,5455.680,116.385199,40.000484,91.5
5457.680,17.975,3.000,15.000,1694514659.265,1.000,5456.680,116.385199,40.000484,91.5
5458.680,17.975,3.000,15.000,1694514659.292,1.000,5457.680,116.38

In [21]:
est_fd = open('./EvaluationRun01.csv', 'w')

cnt = 0
init_gps_tow = gt_gnss_unit_list[0].gps_tow

r = requests.get(server + trialname + '/estimates')
for i, line in enumerate(r.text.splitlines()):
    if i in {0, 1}:
        continue
    s = parse(estfmt, line)
    ts = float(s.named['pts'])
    lng, lat, altitude = [float(x) for x in s.named['pos'].split(',')]
    
    if cnt == 0:
        ts = init_gps_tow
    else:
        ts = init_gps_tow + cnt
    est_fd.write('%.3f,%.6f,%.6f,%.1f\n' % (ts, lat, lng, altitude))
    cnt += 1
    
est_fd.close()

In [22]:
r = requests.get(server + trialname + '/estimates')
print(len(r.text.splitlines()))

5497


In [23]:
print(parse(estfmt, r.text.splitlines()[-1]).named)

{'pts': 5529.68, 'c': 1694514661.708, 'h': 1.0, 's': 15.0, 'pos': '116.385128,40.002315,91.5'}


In [24]:
print(gt_gnss_unit_list[0].gps_tow)
print(gt_gnss_unit_list[0])

354293.0
34.815,116.385107,40.002201,302.300,0.300


## 评估

In [25]:
ts_estimate_loc_map = {}

r = requests.get(server + trialname + '/estimates')
for i, line in enumerate(r.text.splitlines()):
    if i in {0, 1}:
        continue
    s = parse(estfmt, line)
    ts = int(float(s.named['pts']))
    loc_str = s.named['pos']
    lng, lat = float(loc_str.split(',')[0]), float(loc_str.split(',')[1])
    lng, lat = geo_util.wgs84_to_gcj02(lng, lat)
    ts_estimate_loc_map[ts] = [lng, lat]
    
ts_list = sorted(ts_estimate_loc_map.keys(), reverse=False)
with open('offline_estimate.csv', 'w') as fd:
    fd.write('ts,lng,lat\n')
    for i, ts in enumerate(ts_list):
        lng, lat = ts_estimate_loc_map[ts]
        if i > 0:
            ts = ts - ts_list[0]
        fd.write('%d,%.6f,%.6f\n' % (ts, lng, lat))

In [26]:
ts_gnss_loc_map = {}

for i, gnss in enumerate(gt_gnss_unit_list):
    ts = int(round(gnss.app_ts, 0))
    lng, lat = gnss.lng_gcj02, gnss.lat_gcj02
    ts_gnss_loc_map[ts] = [lng, lat]
    
ts_list = sorted(ts_gnss_loc_map.keys(), reverse=False)
with open('offline_gnss.csv', 'w') as fd:
    fd.write('ts,lng,lat\n')
    for i, ts in enumerate(ts_list):
        lng, lat = ts_gnss_loc_map[ts]
        if i > 0:
            ts = ts - ts_list[0]
        fd.write('%d,%.6f,%.6f\n' % (ts, lng, lat))

In [27]:
len(ts_gnss_loc_map)

2968

In [28]:
dist_err_list = []

for ts, (lng, lat) in ts_estimate_loc_map.items():
    if ts not in ts_gnss_loc_map and ts + 1 in ts_gnss_loc_map:
        dist_err = geo_util.distance(lng, lat, 
                                     ts_gnss_loc_map[ts + 1][0], ts_gnss_loc_map[ts + 1][1])
        dist_err_list.append(dist_err)

In [29]:
print(np.percentile(dist_err_list, 25))
print(np.percentile(dist_err_list, 50))
print(np.percentile(dist_err_list, 75))

20.3874145
22.472451
49.09546949999999


In [30]:
dist_err_list

[18.302378, 75.718488, 22.472451]

In [31]:
cnt = 0
init_gps_tow = gt_gnss_unit_list[0].gps_tow

tow_estimate_locs = []

r = requests.get(server + trialname + '/estimates')
for i, line in enumerate(r.text.splitlines()):
    if i in {0, 1}:
        continue
    s = parse(estfmt, line)
    ts = float(s.named['pts'])
    lng, lat, altitude = [float(x) for x in s.named['pos'].split(',')]
    
    if cnt == 0:
        ts = init_gps_tow
    else:
        ts = init_gps_tow + cnt
    cnt += 1
    tow_estimate_locs.append([lng, lat, ts])
    
gt_gps_tows = set([u.gps_tow for u in gt_gnss_unit_list])

In [32]:
posi_unit_list = []
posi_unit_map = {}

with open('./trials2023/POSI_testingData.txt', 'r') as fd:
    for line in fd:
        if line.startswith('POSI'):
            unit = posiUnit(line.strip())
            posi_unit_list.append(unit)
            posi_unit_map[unit.gps_tow] = unit
            
print(len(posi_unit_list))

5494


In [36]:
errs = []
for posi_unit, (lng, lat, ts) in zip(posi_unit_list, tow_estimate_locs):
    if ts in gt_gps_tows:
        continue
    err = geo_util.distance(posi_unit.lng, posi_unit.lat, lng, lat)
    errs.append(err)
    
print(len(errs))
print(np.percentile(errs, 25))
print(np.percentile(errs, 50))
print(np.percentile(errs, 75))

2526
27.6935805
54.8975455
73.80828825


In [34]:
errs = []
for posi_unit, (lng, lat, ts) in zip(posi_unit_list, tow_estimate_locs):
    err = geo_util.distance(posi_unit.lng, posi_unit.lat, lng, lat)
    errs.append(err)
    
print(len(errs))
print(np.percentile(errs, 25))
print(np.percentile(errs, 50))
print(np.percentile(errs, 75))

5494
2.8409955
6.0677520000000005
51.85888275


In [37]:
print(len(errs))
print(np.percentile(errs, 25))
print(np.percentile(errs, 50))
print(np.percentile(errs, 75))

2526
27.6935805
54.8975455
73.80828825
